In [1]:
import pandas as pd 
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from datasets import load_dataset
import os
os.environ["TF_USE_LEGACY_KERAS"] ="1"
from sentence_transformers import SentenceTransformer

In [81]:
ar_df = load_dataset("mohres/The_Arabic_E-Book_Corpus")
train_split = ar_df['train']
ar_df = train_split.to_pandas()


In [82]:
my_df = pd.read_csv("C:/Users/aliel/Downloads/goodreads_library_export (1).csv", on_bad_lines='warn')
book_df = pd.read_csv("C:/Users/aliel/Downloads/Goodreadss Books.csv")

In [83]:
book_df.columns = book_df.columns.str.lower()
required_cols = ['title', 'author', 'description', 'genres']
book_df_clean = book_df[required_cols]


In [84]:
ar_df_renamed = ar_df.rename(columns={
    'text': 'description',
    'category': 'genres'
    
    })

ar_df_clean = ar_df_renamed[['title', 'author', 'description', 'genres']]

In [89]:
master_df = pd.concat([book_df_clean, ar_df_clean], ignore_index=True)
master_df = master_df.fillna('')

In [91]:
master_df = master_df.drop_duplicates(subset=['title'], keep='first')
master_df = master_df.reset_index(drop=True)
len(master_df)

18642

In [93]:
master_df.tail(10)

,title,author,description,genres
18632,باطن الأرض: فجوة عملاقة تمتد عبر الكوكب من قطب...,إدوارد بيدج ميتشل,باطن الأرض\n\nباطن الأرض\n\nفجوة عملاقة تمتد ع...,science.fiction
18633,بين الجزر والمد: صفحات في اللغة والآداب والفن ...,مي زيادة,بين الجزر والمد\n\nبين الجزر والمد\n\nصفحات في...,literature
18634,عزة بنت الخليفة: رواية تمثيلية ذات فصل واحد,إبراهيم رمزي,عزة بنت الخليفة\n\nعزة بنت الخليفة\n\nرواية تم...,plays
18635,المهلهل سيد ربيعة,محمد فريد أبو حديد,المهلهل سيد ربيعة\n\nالمهلهل سيد ربيعة\n\nتألي...,novels
18636,حكايات المهجر,عبد المسيح حداد,حكايات المهجر\n\nحكايات المهجر\n\nتأليف\nعبد ا...,"literature,travel.literature"
18637,لامرتين,إلياس أبو شبكة,لامرتين\n\nلامرتين\n\nتأليف\nإلياس أبو شبكة\n\...,biographies
18638,دليل الملاعق الفضية,روبرت بار,دليل الملاعق الفضية\n\nدليل الملاعق الفضية\n\n...,detective.fiction
18639,كلمات في مبادئ علم الأخلاق,محمد عبد الله دراز,كلمات في مبادئ علم الأخلاق\n\nكلمات في مبادئ ع...,"philosophy,social.sciences"
18640,بابا عَبْدُ الله وَالدَّرْوِيش,كامل كيلاني,بابا عَبْدُ الله وَالدَّرْوِيش\n\nبابا عَبْدُ ...,children.stories
18641,لغز جرس التصويت,إلين ويلكنسون,لغز جرس التصويت\n\nلغز جرس التصويت\n\nتأليف\nإ...,detective.fiction


In [95]:
master_df['content'] = master_df['title'] + ' ' + master_df['author'] + ' ' + master_df['description'] + ' ' + master_df['genres']

In [97]:
print(master_df[['title', 'content']].tail(10))

                                                   title  \
18632  باطن الأرض: فجوة عملاقة تمتد عبر الكوكب من قطب...   
18633  بين الجزر والمد: صفحات في اللغة والآداب والفن ...   
18634        عزة بنت الخليفة: رواية تمثيلية ذات فصل واحد   
18635                                  المهلهل سيد ربيعة   
18636                                      حكايات المهجر   
18637                                            لامرتين   
18638                                دليل الملاعق الفضية   
18639                         كلمات في مبادئ علم الأخلاق   
18640                     بابا عَبْدُ الله وَالدَّرْوِيش   
18641                                    لغز جرس التصويت   

                                                 content  
18632  باطن الأرض: فجوة عملاقة تمتد عبر الكوكب من قطب...  
18633  بين الجزر والمد: صفحات في اللغة والآداب والفن ...  
18634  عزة بنت الخليفة: رواية تمثيلية ذات فصل واحد إب...  
18635  المهلهل سيد ربيعة محمد فريد أبو حديد المهلهل س...  
18636  حكايات المهجر عبد المسيح حداد حكايات 

In [99]:
model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

In [100]:
embeddings = model.encode(master_df['content'].to_list(), show_progress_bar=True)

Batches:   0%|          | 0/583 [00:00<?, ?it/s]

In [151]:
def get_recommendations(query, master_df, model, embeddings):
    query_embeddings = model.encode([query])

    scores = cosine_similarity(query_embeddings, embeddings)[0]

    top_indicies = scores.argsort()[-10:][::-1]
    print("Here are the top 10 semantically similar books:")
    for idx in top_indicies:
        title = master_df.iloc[idx]['title']
        score = scores[idx]
        print(f"{score:.4f} - {title}")

In [153]:
book = "The saga of Arthur Bandini"
get_recommendations(book, master_df, model, embeddings)

Here are the top 10 semantically similar books:
0.5453 - Cyclops / Alcestis / Medea
0.5452 - The Charterhouse of Parma
0.5359 - The War of Don Emmanuel's Nether Parts
0.5216 - Cyrano de Bergerac (Cliffs Notes
0.5213 - The King and the Corpse: Tales of the Soul's Conquest of Evil
0.5158 - Olympos
0.5139 - Der Medicus. Sonderausgabe.
0.5091 - The Three Theban Plays: Antigone, Oedipus the King, Oedipus at Colonus
0.5087 - الثابت
0.5058 - Jacques and His Master: An Homage to Diderot in Three Acts


In [147]:
def get_profile_recommendations(my_df, master_df, model, embeddings, min_rating=4):

    liked_books = my_df[my_df['My Rating'] >= min_rating]['Title'].tolist()
    
    if not liked_books:
        print(f"No books found with a rating of {min_rating} or higher. Try a lower rating.")
        return

    print(f"Found {len(liked_books)} books you loved. Examples: {liked_books[:3]}")

    user_book_vectors = model.encode(liked_books, show_progress_bar=True)
    
    user_profile_vector = user_book_vectors.mean(axis=0)

    profile_scores = cosine_similarity([user_profile_vector], embeddings)[0]
    profile_top_indicies = profile_scores.argsort()[-20:][::-1]

    read_books_set = set(my_df['Title'].str.lower())

    count = 0
    for idx in profile_top_indicies:
            title = master_df.iloc[idx]['title']
            score = profile_scores[idx]

            if title.lower() in read_books_set:
                continue
            
            print(f"{score:.4f} - {title}")
            count += 1
            if count >= 10:
                break

In [149]:
get_profile_recommendations(my_df, master_df, model, embeddings)

Found 171 books you loved. Examples: ['الجوع', 'أحلام من بنكر هيل  (The Saga of Arthur Bandini, #4)', 'اسأل الغبار (Arturo Bandini, #3)']


Batches:   0%|          | 0/6 [00:00<?, ?it/s]

0.7884 - 
0.6912 - مناوشات
0.6758 - مجددون ومجترون
0.6593 - العواصف
0.6580 - غابة الحق
0.6554 - المقالات الأدبية
0.6539 - مغامرة منزل آبي جرينج
0.6537 - واحد منا
0.6531 - لذكراك
0.6526 - سيدة القصور
